### Calculate F-score given a gold and predicted list (or nbest predicted list)

In [37]:
import cc_utils
import importlib
importlib.reload(cc_utils)
from cc_utils import *

In [5]:
gold = load_gold("/Users/hsajjad/Work/work/comparable-corpus/fr-en.training.gold")

In [6]:
pred_f = "/Users/hsajjad/Work/work/comparable-corpus/predictions-1best"
pred = load_prediction(pred_f)

In [10]:
# calculate precision, recall and fmeasure
score = calculateScore(pred, gold)
print (score)

p = 0.023474109330057304, r = 0.7023992956196347, f = 0.04542995444191344


In [23]:
# to calculate score of nbest list. This is to look at what is the best recall in our nbest list
pred_nbest = nbest_oracle("/Users/hsajjad/Work/work/comparable-corpus/fr-en-parallel-top10-candidates")

In [24]:
nbest = 10
nbest_score = calculateOracleScore(pred_nbest, gold, nbest)
print (nbest_score)

p = 0.027854815098170477, r = 0.8334800792427911, f = 0.053908029612756266


### Load machine translation outputs to find the best candidate sentences from a list of nbest candidates 

In [29]:
### load MT output
### Format of file is French sentence tab English translation

mt_pairs = {}
mt_idx = {}

train_mt="/Users/hsajjad/Work/work/comparable-corpus/mt/train-fr-en-mt-out" 
load_mt(train_mt, mt_pairs, mt_idx)
len(mt_pairs)

270730

In [30]:
# load MT output for train
### Format of file is French sentence tab English translation

train_mt="/Users/hsajjad/Work/work/comparable-corpus/mt/train-fr-en-mt-out"
train_mt_pairs = {}
train_mt_idx = {}
load_mt(train_mt, train_mt_pairs, train_mt_idx)

### save nbest file with bleu scores on every line
train_nbest = "/Users/hsajjad/Work/work/comparable-corpus/fr-en-parallel-top10-candidates"
train_nbest_bleu = "/Users/hsajjad/Work/work/comparable-corpus/fr-en-parallel-top10-candidates-bleu"
nbest_bleu_save(train_nbest, train_nbest_bleu, train_mt_pairs)


In [38]:
# load file with bleu score and choose the candidate with the hightest bleu
train_nbest_bleu = "/Users/hsajjad/Work/work/comparable-corpus/fr-en-parallel-top10-candidates-bleu"
dic = getBestBleu(train_nbest_bleu)

In [41]:
# select candidates with bleu greater than a certain threshold
threshold = 0.55
train_final = filter_on_bleu(dic, threshold)
calculateScore(train_final, gold)

'p = 0.7902658089573977, r = 0.7165969623596742, f = 0.7516305916305915'

In [236]:
# load MT output for test
test_mt="/Users/hsajjad/Work/work/comparable-corpus/mt/test-fr-en-mt-out"
test_mt_pairs = {}
test_mt_idx = {}
load_mt(test_mt, test_mt_pairs, test_mt_idx)
print (len(test_mt_pairs))


275683


In [ ]:
## save nbest file with bleu scores on every line
test_nbest = "/Users/hsajjad/Work/work/comparable-corpus/fr-en-parallel-top10-candidates-testset"
test_nbest_bleu = "/Users/hsajjad/Work/work/comparable-corpus/fr-en-parallel-top10-candidates-testset-bleu"
nbest_bleu_save(test_nbest, test_nbest_bleu, test_mt_pairs)

In [225]:
test_nbest_bleu = "/Users/hsajjad/Work/work/comparable-corpus/fr-en-parallel-top10-candidates-testset-bleu"
test_pred = getBestBleu(test_nbest_bleu)
len(test_pred)

276833

In [259]:
threshold = 0.52
test_final = filter_on_bleu(test_pred, threshold)

### Some code to write files in BUCC submission format

In [260]:
def bucc_format(st, tag):
    st_rest = 9 - len(st)
    for i in range(st_rest):
        st = '0' + st
    st = tag + st
    return st
        
def output_format(pred_dic):
    format_dic = {}
    for key,value in pred_dic.items():
        fr, en = key.split("\t")
        fr = bucc_format(fr, "fr-")
        en = bucc_format(en, "en-")
        format_dic[fr + "\t" + en] = 0
    return format_dic

In [261]:
formatted_out = output_format(test_final)
test_output = open ("/Users/hsajjad/Work/work/comparable-corpus/submission/test-embedding-mt-0.52", 'w', encoding='utf-8')
for key,value in formatted_out.items():
    test_output.write(key)
    test_output.write("\n")
test_output.close()


In [281]:
#### QE task
qeFile = "/Users/hsajjad/Work/work/comparable-corpus/fr-en-qe-predictions-train.sentids+qescores-filtered"
qeFile = "/Users/hsajjad/Work/work/comparable-corpus/eval-qe-bleu"
qeFile = "/Users/hsajjad/Work/work/comparable-corpus/eval-qe-0.5"

qpred ={}

qf = open(qeFile, 'r', encoding="utf-8")
for line in qf:
    line = line.rstrip()
    fr_id, en_id = line.split("\t")
    fr_id = str(int(fr_id.replace('fr-','')))
    en_id = str(int(en_id.replace('en-','')))
    qpred[fr_id + "\t" + en_id] = 0

In [283]:
calculateScore(qpred, gold)

5894 2845 3192
0.6744478773315025 0.6486902927580893 0.6613183730715287


In [ ]:
formatted_out = output_format(test_final)
test_output = open ("/Users/hsajjad/Work/work/comparable-corpus/submission/test-embedding-qe", 'w', encoding='utf-8')
for key,value in formatted_out.items():
    test_output.write(key)
    test_output.write("\n")
test_output.close()
